### Automated backwards optimization

Automate the discovery of strategy rules.

In [2]:
import datetime
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule
from itertools import product, combinations

import numpy as np

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = "weekly"

# FRAMEWORK = 'dask'
FRAMEWORK = "pandas"

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/forward_opt_results.json"
BACKWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/backward_opt_results.json"

# The minimum number of trades in a period for it to be considered.
MIN_TRADES_REQUIRED = 10

In [3]:
metrics_options = ["adx", "ppo", "stochastic"]


if TIMEFRAME == "daily":
    size_options = [8, 16, 32]
    path = "dataset/daily/"
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx8",
        "adx16",
        "adx32",
        "ppo8",
        "ppo16",
        "ppo32",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]
elif TIMEFRAME == "weekly":
    size_options = [3, 6, 9]
    path = "dataset/weekly/"
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx3",
        "adx6",
        "adx9",
        "ppo3",
        "ppo6",
        "ppo9",
        "stochastic3",
        "stochastic6",
        "stochastic9",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]

if FRAMEWORK == "pandas":
    d = pd
    f = f"{path}/all_results.csv"
elif FRAMEWORK == "dask":
    d = dd
    f = f"{path}/*.csv"

FULL_DF = d.read_csv(
    f,
    names=column_names,
)


def get_full_df():
    return d.read_csv(
        f,
        names=column_names,
    )

In [4]:
get_full_df()

,uid,ticker,date,price_open,maperiod,rsi_open_period,adx3,adx6,adx9,ppo3,ppo6,ppo9,stochastic3,stochastic6,stochastic9,price_sell,days_ago_close_period,rsi_close_period
0,48c17ecc-c05c-11eb-8547-95fa4750c8c5,ABT,1998-12-10,12.32,35,2,55.428198,24.777688,20.590541,11.332148,11.039301,9.796907,51.086957,60.635452,76.149917,12.93,0,61.107269
1,48c184da-c05c-11eb-8547-95fa4750c8c5,ABT,1998-10-01,11.13,30,2,34.046867,22.352198,23.652706,6.676282,6.655369,6.145646,61.615041,77.178423,77.178423,11.67,0,64.068306
2,48c1a488-c05c-11eb-8547-95fa4750c8c5,ABT,1999-01-21,12.26,30,4,60.347924,26.885536,19.231565,4.499533,5.571080,5.589020,48.482984,44.457008,44.124282,12.50,0,43.644592
3,48c17ecc-c05c-11eb-8547-95fa4750c8c5,ABT,1998-12-10,12.32,35,2,55.428198,24.777688,20.590541,11.332148,11.039301,9.796907,51.086957,60.635452,76.149917,12.99,1,64.866750
4,48c184da-c05c-11eb-8547-95fa4750c8c5,ABT,1998-10-01,11.13,30,2,34.046867,22.352198,23.652706,6.676282,6.655369,6.145646,61.615041,77.178423,77.178423,11.99,1,77.572977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42591,f63491be-c05d-11eb-8548-95fa4750c8c5,MO,2020-09-10,39.43,55,2,51.966974,37.774219,25.053262,1.285612,1.363133,0.626082,41.304850,58.722201,67.418177,37.22,1,27.583623
42592,f63491be-c05d-11eb-8548-95fa4750c8c5,MO,2020-09-10,39.43,55,2,51.966974,37.774219,25.053262,1.285612,1.363133,0.626082,41.304850,58.722201,67.418177,39.04,2,69.312192
42593,f63491be-c05d-11eb-8548-95fa4750c8c5,MO,2020-09-10,39.43,55,2,51.966974,37.774219,25.053262,1.285612,1.363133,0.626082,41.304850,58.722201,67.418177,38.10,3,43.449716
42594,f63491be-c05d-11eb-8548-95fa4750c8c5,MO,2020-09-10,39.43,55,2,51.966974,37.774219,25.053262,1.285612,1.363133,0.626082,41.304850,58.722201,67.418177,37.09,4,24.114173


In [5]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


FORWARD_OPT_JSON = get_forward_opt_json()


def get_pair_entry(start_date, end_date):
    return (
        f"{start_date.strftime('%y-%m')}-to-{end_date.strftime('%y-%m')}",
        start_date,
        end_date,
    )


def filter_df_by_dates(df, start_date, end_date):
    return df[
        (df["date"] >= start_date.strftime("%Y-%m-%d"))
        & (df["date"] <= end_date.strftime("%Y-%m-%d"))
    ]


def get_filtered_df_by_key(key, range_name, func_type):
    optimal_args = FORWARD_OPT_JSON[key][range_name][func_type]

    all_date_ranges = generate_date_ranges()
    _, start_date, end_date = all_date_ranges[key][range_name]

    df = get_full_df()
    df = filter_df_by_dates(df, start_date, end_date)

    for k, v in optimal_args.items():
        df = df[df[k] == v]

    return df


def generate_date_ranges():
    resulting_dict = {}
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE):
        resulting_dict[end_date.strftime("%y-%m")] = {
            "6-month": get_pair_entry(end_date + relativedelta(months=-6), end_date),
            "12-month": get_pair_entry(end_date + relativedelta(months=-12), end_date),
            "max": get_pair_entry(datetime.datetime(1998, 1, 1), end_date),
        }

    return resulting_dict


# get_filtered_df_by_key('13-10', '6-month', 'mean')

In [6]:
keys = [
    end_date.strftime("%y-%m")
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE)
]
range_values = [
    "6-month",
    "12-month",
]
func_values = ["mean", "order"]

In [7]:
def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


BACKWARD_OPT_JSON = get_backward_opt_json()


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False

    if range_val not in BACKWARD_OPT_JSON[key]:
        return False

    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False

    return True

In [9]:
def save_pair_result(key, range_val, func_val, optimal_res):
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        backward_opt_json = json.loads(f.read())

    backward_opt_json[key] = backward_opt_json.get(key, {})
    backward_opt_json[key][range_val] = backward_opt_json[key].get(range_val, {})
    backward_opt_json[key][range_val][func_val] = optimal_res

    with open(BACKWARD_OPT_RESULTS_JSON, "w") as f:
        f.write(json.dumps(backward_opt_json))

In [10]:
def mean_optimal_func(tmp_df):
    max_value = tmp_df["perc_change"].mean()

    if FRAMEWORK == "dask":
        max_value = max_value.compute()

    return max_value


def order_optimal_func(tmp_df):
    l = []
    for _, perc_change in tmp_df.sort_values(["date"])["perc_change"].iteritems():
        l.append(perc_change)

    results = []
    for i in range(len(l)):
        res = 1
        for j in range(i, len(l)):
            res += (res * l[j]) / 100
        results.append(res)

    return sum(results) / len(results)

In [11]:
def single_compute_optimal_res(df, func):
    upper_barrier_results = []
    lower_barrier_results = []
    singles = [f"{x[0]}{x[1]}" for x in list(product(metrics_options, size_options))]

    for indicator in singles:
        max_val, min_val = int(df[indicator].max()), int(df[indicator].min())

        for barrier in range(min_val, max_val + 2, 2):
            filtered_df = df[df[indicator] <= barrier]
            if filtered_df.shape[0] >= MIN_TRADES_REQUIRED:
                outcome = func(filtered_df)
                if outcome and outcome not in [np.nan]:
                    upper_barrier_results.append([indicator, barrier, float(outcome)])

            filtered_df = df[df[indicator] >= barrier]
            if filtered_df.shape[0] >= MIN_TRADES_REQUIRED:
                outcome = func(filtered_df)
                if outcome and outcome not in [np.nan]:
                    lower_barrier_results.append([indicator, barrier, float(outcome)])

    upper_barrier_df = pd.DataFrame(
        np.array(upper_barrier_results), columns=["indicator", "barrier", "value"]
    )
    upper_barrier_res = upper_barrier_df[
        upper_barrier_df["value"] == upper_barrier_df["value"].max()
    ]

    lower_barrier_df = pd.DataFrame(
        np.array(lower_barrier_results), columns=["indicator", "barrier", "value"]
    )
    lower_barrier_res = lower_barrier_df[
        lower_barrier_df["value"] == lower_barrier_df["value"].max()
    ]

    if upper_barrier_df["value"].max():
        best_type = "upper"
        best_indicator = upper_barrier_res.iloc[0]["indicator"]
        best_barrier = int(upper_barrier_res.iloc[0]["barrier"])
    else:
        best_type = "lower"
        best_indicator = lower_barrier_res.iloc[0]["indicator"]
        best_barrier = int(lower_barrier_res.iloc[0]["barrier"])

    return {
        "upper": {
            "indicator": upper_barrier_res.iloc[0]["indicator"],
            "barrier": int(upper_barrier_res.iloc[0]["barrier"]),
        },
        "lower": {
            "indicator": lower_barrier_res.iloc[0]["indicator"],
            "barrier": int(lower_barrier_res.iloc[0]["barrier"]),
        },
        "best": {
            "type": best_type,
            "indicator": best_indicator,
            "barrier": best_barrier,
        },
    }


def double_compute_optimal_res(df, func):
    pass


def compute_optimal_res(df):
    result = {}
    for name, func in [("mean", mean_optimal_func), ("order", order_optimal_func)]:
        result[name] = {
            "single": single_compute_optimal_res(df, func),
            #             'double': double_compute_optimal_res(df, func),
        }

    return result

In [12]:
all_pairs = list(product(keys, range_values, func_values))

unparsed_pairs = [
    (key, range_val, func_val)
    for key, range_val, func_val in all_pairs
    if not is_pair_parsed(key, range_val, func_val)
]

unparsed_pairs

[]

In [31]:
with tqdm.tqdm(total=len(unparsed_pairs)) as pbar:

    for key, range_val, func_val in unparsed_pairs:
        try:
            df = get_filtered_df_by_key(key, range_val, func_val)

            df["perc_change"] = (
                (df["price_sell"] - df["price_open"]) / df["price_open"] * 100
            )
            df = df.drop(["price_sell", "price_open"], axis=1)
            print(
                f"Key in forward opt json {key} - {range_val} - {func_val} - {df.shape[0]}"
            )

        except KeyError:
            print(f"Key NOT in forward opt json {key} - {range_val} - {func_val}")
            pbar.update()
            continue

        size = df.shape[0] if FRAMEWORK == "pandas" else df.shape[0].compute()

        if size < MIN_TRADES_REQUIRED:
            print(f"Not enough trades for {key} - {range_val} - {func_val}")
            optimal_res = None
        else:
            optimal_res = compute_optimal_res(df)

        # 06-06 - 6-month - mean - 10
        print(f"{key} - {range_val} - {func_val}")
        print(optimal_res)

        save_pair_result(key, range_val, func_val, optimal_res)
        pbar.update()

  0%|          | 0/996 [00:00<?, ?it/s]

Key in forward opt json 00-04 - 6-month - mean - 132


  0%|          | 1/996 [00:17<4:51:53, 17.60s/it]

00-04 - 6-month - mean
{'mean': {'single': {'upper': {'indicator': 'stochastic8', 'barrier': 9}, 'lower': {'indicator': 'ppo8', 'barrier': 22}, 'best': {'type': 'upper', 'indicator': 'stochastic8', 'barrier': 9}}}, 'order': {'single': {'upper': {'indicator': 'stochastic32', 'barrier': 75}, 'lower': {'indicator': 'ppo8', 'barrier': 14}, 'best': {'type': 'upper', 'indicator': 'stochastic32', 'barrier': 75}}}}
Key in forward opt json 00-04 - 6-month - order - 244


  0%|          | 2/996 [00:35<4:57:56, 17.98s/it]

...

20-12 - 6-month - order
{'mean': {'single': {'upper': {'indicator': 'ppo32', 'barrier': 0}, 'lower': {'indicator': 'ppo8', 'barrier': 31}, 'best': {'type': 'upper', 'indicator': 'ppo32', 'barrier': 0}}}, 'order': {'single': {'upper': {'indicator': 'adx8', 'barrier': 34}, 'lower': {'indicator': 'stochastic16', 'barrier': 57}, 'best': {'type': 'upper', 'indicator': 'adx8', 'barrier': 34}}}}
Key in forward opt json 20-12 - 12-month - mean - 909


100%|█████████▉| 995/996 [6:41:56<00:23, 23.35s/it]

20-12 - 12-month - mean
{'mean': {'single': {'upper': {'indicator': 'ppo32', 'barrier': -4}, 'lower': {'indicator': 'ppo16', 'barrier': 35}, 'best': {'type': 'upper', 'indicator': 'ppo32', 'barrier': -4}}}, 'order': {'single': {'upper': {'indicator': 'adx16', 'barrier': 32}, 'lower': {'indicator': 'stochastic16', 'barrier': 48}, 'best': {'type': 'upper', 'indicator': 'adx16', 'barrier': 32}}}}
Key in forward opt json 20-12 - 12-month - order - 917


100%|██████████| 996/996 [6:42:24<00:00, 24.24s/it]

20-12 - 12-month - order
{'mean': {'single': {'upper': {'indicator': 'ppo8', 'barrier': -1}, 'lower': {'indicator': 'ppo8', 'barrier': 31}, 'best': {'type': 'upper', 'indicator': 'ppo8', 'barrier': -1}}}, 'order': {'single': {'upper': {'indicator': 'ppo8', 'barrier': 11}, 'lower': {'indicator': 'stochastic32', 'barrier': 49}, 'best': {'type': 'upper', 'indicator': 'ppo8', 'barrier': 11}}}}
